In [18]:
import folium
from PIL import Image,ExifTags
import glob,os

startLoc= [60.3913, 5.3221] # Bergen, Norway

style = {'color': '#F7DC6F', 'weight':'1'}
m = folium.Map(
    startLoc,
    zoom_start=10,
    )
         
folium.Circle(radius=500,location= startLoc,color='crimson',fill=False).add_to(m)

tooltip = "Click Here For More Info"

def dms2dd(pos, direction):
    degrees, minutes, seconds=pos
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    # this puts stuff in the North Sea!!!
    #if direction == 'E' or direction == 'S':
    #    dd *= -1
    return dd;

# extract EXIF data
sourcePath="M:/Camera/pics/*.jpg"

# to reduce spelling mistakes
GPSLat="GPSLatitude"
GPSLatRef=GPSLat+"Ref"
GPSLon="GPSLongitude"
GPSLonRef=GPSLon+"Ref"
try:

   # picInfo=[] # filename,date_time,lat,long
    
    print("Scanning images")
    
    for infile in glob.glob(sourcePath):
        with Image.open(infile) as img:
            exif_data = img._getexif()

            if exif_data is not None:

                exif = {
                    ExifTags.TAGS[k]: v
                    for k, v in exif_data.items()
                    if k in ExifTags.TAGS
                }

                if "GPSInfo" in exif:

                    imgName=os.path.basename(infile)
                    
                    gpsinfo = {}
                    for key in list(exif['GPSInfo'].keys()):
                        decode = ExifTags.GPSTAGS.get(key, key)
                        gpsinfo[decode] = exif['GPSInfo'][key]
                        
                    if "GPSLatitude" in gpsinfo:
                        # extract location and add to row
                        
                        lat = dms2dd(gpsinfo[GPSLat],gpsinfo[GPSLatRef])
                        lon = dms2dd(gpsinfo[GPSLon], gpsinfo[GPSLonRef])
                        # GPSlong/lat degrees,minutes,centiseconds
                    
                        dt=exif["DateTimeOriginal"]

                        marker = folium.Marker(
                            location=    [lat,lon],
                            popup=f"<stong>{imgName}\n{round(lat,3)},{round(lon,3)}\n{dt}</stong>",
                            tooltip=tooltip)
                        
                        marker.add_to(m)
                        # row=(imgName,exif["DateTimeOriginal"],lat,lon)

except Exception as e:
    print("EXCEPTION",e)


# add a line
#coords=[(53.796621, -0.34482),(53.896621, -0.35482)]

#aline=folium.PolyLine(locations=coords,weight=2,color = 'blue')
#aline.add_to(m)

m

Scanning images
